In [19]:
# Load Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

class eigenestimator_decomp:

    def __init__(self, basis_functions, data, deltat, eigvalue, neg_point, pos_point):
        # self.basis_functions: basis functions for estimating eigenfunctions 
        # self.data: brusselator trajectories
        # self.deltat: time difference between each datapoint
        # self.neg_point: the point on the plot that is required to be negative
        # self.p0s_point: the point on the plot that is required to be positive
        self.basis_functions = basis_functions
        self.data = data
        self.deltat = deltat
        self.eigvalue = eigvalue
        self.neg_point = neg_point
        self.pos_point = pos_point

    def estimate(self):
        n_basis = len(self.basis_functions(self.data[0][0, :])) # dimension of basis function output of one data point
        self.gamma = np.zeros((n_basis, n_basis), dtype = complex)
        for i in range(len(self.data)):
            # self.p - number of time points / number of rows
            # self.n - number of variables / number of columns
            self.p, self.n = self.data[i].shape
            # Prepare two matrices with a delta t difference
            self.x, self.y = self.data[i][:-1], self.data[i][1:]

            # Calculate the term multiplied by b
            # G(x) --> [g1(x), g2(x), ... , gq(x)]T
            # G(x(1)) --> g1([x1(1), x2(1), x3(1)]), g2([x1(1), x2(1), x3(1)])
            # A(x(t)) --> [G(x(t)) - e^{c*deltat}G(x(t))], c is a constant  

            G_x = np.array([self.basis_functions(x) for x in self.x]) 
            G_y = np.array([self.basis_functions(y) for y in self.y])
            Delta = (G_y - np.exp(self.eigvalue*self.deltat) * G_x) / self.deltat

            self.gamma += (Delta.conj().T @ Delta) / self.x.shape[0] / len(self.data) 

        # Find eigenvalues and eigenvectors of the matrix
        self.eigenvalues, self.eigenvectors = np.linalg.eigh(self.gamma) # eigh() used for symmetric matrices, faster / also orders the eigenvalues already

        # Determine coefficients for the linear combination by testing each eigenvalue from least to greatest. Ensure the opposite sign criteria is met. 
        for i in range(n_basis):
            self.coefficients = self.eigenvectors[:, i]
            self.estimated_function = lambda x: np.dot(self.coefficients, self.basis_functions(x))
            if True: # np.real(self.estimated_function(self.neg_point) * self.estimated_function(self.pos_point)) < 0:
                break
        self.error = np.real(np.vdot(self.coefficients, self.gamma @ self.coefficients)) # np.vdot() conjugates first vector and does dot product
    
    def plot(self):
        fig, ax = plt.subplots() # subplots(fig, ax) defaults to 1 axis
        x = np.linspace(0.01, 6.0, 100)
        y = np.linspace(0.01, 6.0, 100)
        X, Y = np.meshgrid(x, y) # for all i in len(y) and all j in len(x) (X[i, j], Y[i, j])

        # Define function
        func = self.estimated_function

        # Reshape to match meshgrid
        Z = np.real(np.array([[func(np.array([xx, yy])) for xx in x] for yy in y]))

        # Remove the unsampled areas
        no_data = (X <= 0.2) | (Y <= 0.1) | ((X + Y) >= 7) | ((X + Y) <= 2)
        Z[no_data] = np.nan

        # Plot
        Zmax = np.nanmax(np.abs(Z))
        levels = np.linspace(-Zmax, Zmax, 31)
        cmap = plt.get_cmap('coolwarm')
        norm = mcolors.BoundaryNorm(levels, cmap.N)
        contour = ax.pcolormesh(X, Y, Z, cmap='coolwarm', norm=norm)
        ax.set_xlabel(r'$x_1$'), ax.set_ylabel(r'$x_2$'), fig.colorbar(contour)
        ax.set_title(r'Eigenfunction contour (real part)')
        plt.show()
